# Kotlin Coroutines: suspend functions explained

In this notebook you will learn:
- What a suspend function is and how suspension works.
- How to define and call suspend functions correctly.
- When to use suspend functions (and when not to).
- Practical patterns: switching dispatchers with `withContext`, composing work sequentially vs. in parallel, cancellation, exception handling, and wrapping callbacks.


In [ ]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")


## Imports
We'll import the core coroutines package so that `delay`, `launch`, `async`, etc. are available.


In [ ]:
import kotlinx.coroutines.*
import kotlin.system.measureTimeMillis
import java.io.File


## 1) What is a suspend function?
A `suspend` function is a function that can pause without blocking the underlying thread, and resume later. The pause point is called a suspension point (e.g., `delay`, I/O, waiting on another coroutine, etc.).

Key properties:
- Non-blocking: the thread is free to run other coroutines while waiting.
- Must run in a coroutine: you can only call a `suspend` function from another `suspend` function or a coroutine builder like `runBlocking`, `launch`, or `async`.
- Cancellation-aware: well-behaved suspend functions are cooperative and respond to cancellation promptly.


### Example: a simple suspend function


In [ ]:
suspend fun fetchUserName(): String {
    delay(200) // pretend to do network or database work
    return "Alice"
}

runBlocking {
    val name = fetchUserName()
    println("Hello, $name!")
}


## 2) Suspension vs blocking
Contrast `delay` (suspends without blocking) with `Thread.sleep` (blocks the thread).


In [ ]:
fun blockingSleepExample() {
    val t = measureTimeMillis {
        Thread.sleep(300) // blocks the current thread
    }
    println("Blocking slept for ${'$'}t ms (thread was occupied)")
}

runBlocking {
    val t = measureTimeMillis {
        delay(300) // suspends only the coroutine, thread is free
    }
    println("Suspended for ${'$'}t ms (thread could do other work)")
}

blockingSleepExample()


## 3) Creating and calling suspend functions
Rules of thumb:
- You can call `suspend` functions only from other `suspend` functions or from inside a coroutine builder (`runBlocking`, `launch`, `async`, etc.).
- Prefer small, single-responsibility suspend functions composed together.
- Make them cancellation-friendly (avoid swallowing `CancellationException`; use `ensureActive()` if looping).


### Example: using withContext to switch threads for blocking I/O
Use `withContext(Dispatchers.IO)` to move blocking work off the main/default thread pool.


In [ ]:
suspend fun readFileText(path: String): String = withContext(Dispatchers.IO) {
    File(path).takeIf { it.exists() }?.readText() ?: "<missing>"
}

runBlocking {
    val tmp = kotlin.io.path.createTempFile().toFile()
    tmp.writeText("Hello from disk!")
    println(readFileText(tmp.absolutePath))
}


## 4) Sequential vs parallel composition
Use plain calls for sequential steps; use `async/await` for concurrent, independent work.


In [ ]:
suspend fun fetchUser(): String { delay(200); return "user#42" }
suspend fun fetchPosts(user: String): List<String> { delay(300); return listOf("post-1 of $user", "post-2 of $user") }
suspend fun fetchRecommendations(): List<String> { delay(250); return listOf("rec-1", "rec-2") }

runBlocking {
    val sequential = measureTimeMillis {
        val user = fetchUser()
        val posts = fetchPosts(user) // depends on user → sequential
        val recs = fetchRecommendations() // independent but still sequential here
        println("sequential → posts=${'$'}posts, recs=${'$'}recs")
    }

    val parallel = measureTimeMillis {
        val user = fetchUser()
        val postsDeferred = async { fetchPosts(user) } // can run in parallel
        val recsDeferred = async { fetchRecommendations() }
        val posts = postsDeferred.await()
        val recs = recsDeferred.await()
        println("parallel → posts=${'$'}posts, recs=${'$'}recs")
    }

    println("sequential took ${'$'}sequential ms; parallel took ${'$'}parallel ms")
}


## 5) Cancellation basics for suspend functions
Suspend functions should be cooperative. Many suspend functions (like `delay`) check for cancellation automatically. In loops, use `coroutineContext.ensureActive()` or `yield()` to stay responsive.


In [ ]:
suspend fun cancellableWork() {
    repeat(1000) { i ->
        coroutineContext.ensureActive() // throws if cancelled
        if (i % 100 == 0) println("working… i=${'$'}i")
        delay(10)
    }
}

runBlocking {
    val job = launch { cancellableWork() }
    delay(120) // let it do some work
    println("Cancelling…")
    job.cancelAndJoin()
    println("Cancelled.")
}


## 6) Exception handling with suspend functions
Exceptions thrown from suspend functions propagate like normal. Use `try/catch`, and remember that cancellation uses `CancellationException` (typically let it propagate).


In [ ]:
suspend fun loadData(): String {
    delay(50)
    error("Boom!")
}

runBlocking {
    try {
        loadData()
    } catch (e: CancellationException) {
        println("Cancelled: ${'$'}e") // usually rethrow
    } catch (e: Throwable) {
        println("Failure: ${'$'}e")
    }
}


## 7) Wrapping callback-based APIs with suspend
Use `suspendCancellableCoroutine` to bridge callbacks into the suspend world cleanly.

> Note: this is a schematic example; adapt to your real callback API.


In [ ]:
import kotlin.coroutines.resume
import kotlin.coroutines.resumeWithException
import kotlinx.coroutines.suspendCancellableCoroutine

class FakeButton {
    private var listener: (() -> Unit)? = null
    fun setOnClickListener(l: (() -> Unit)?) { listener = l }
    fun click() { listener?.invoke() }
}

suspend fun awaitClick(btn: FakeButton) = suspendCancellableCoroutine<Unit> { cont ->
    val listener = {
        if (cont.isActive) cont.resume(Unit)
    }
    btn.setOnClickListener(listener)
    cont.invokeOnCancellation { btn.setOnClickListener(null) }
}

runBlocking {
    val btn = FakeButton()
    val job = launch {
        println("Waiting for click…")
        awaitClick(btn)
        println("Clicked!")
    }
    delay(100)
    btn.click() // simulate a click
    job.join()
}


## 8) When to use suspend functions
- I/O-bound operations: network calls, database queries, file I/O.
- Waiting without work: timers, backoff, rate limits, retry delays.
- Orchestrating async steps with structured concurrency (compose with other suspend calls).
- Operations that should be cancellable (user navigated away, timeout, shutdown).

## When NOT to use suspend functions
- Pure CPU-bound work that does not need to suspend or be cancellable: prefer normal functions running on the caller's thread or offload with `withContext(Dispatchers.Default)` only at the call site if needed.
- Very small synchronous operations (simple getters/formatters) — marking them `suspend` adds friction with no benefit.
- Blocking APIs on the main thread: either keep them blocking (no `suspend`) or wrap them and switch to `Dispatchers.IO` using `withContext`.
- Streaming/multi-value emissions over time: consider `Flow` instead of returning large lists or polling in suspend loops.
- "Just in case" design: don't add `suspend` unless there's a real suspension point or a need for cancellation/context.

## Design tips
- Keep suspend APIs single-shot; use `Flow`/`Channel` for streams.
- Use `withContext(Dispatchers.IO)` for blocking I/O; avoid doing blocking work on Default/Main.
- Respect cancellation; avoid catching and swallowing `CancellationException`.
- Consider timeouts with `withTimeout`/`withTimeoutOrNull` where appropriate.
- Document threading expectations and whether the function may switch context.
